In [1]:
!pip install datasets


  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
Using cached multiprocess-0.70.16-py311-none-any.whl (143 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.0
    Uninstalling fsspec-2025.5.0:
      Successfully uninstalled fsspec-2025.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-core 0.12.3 requires pydantic<2.10.0,>=2.7.0, but you have pydantic 2.11.5 which is incompatible.
rag 0.1.0 requires google-adk<0.6.0,>=0.5.0, but you have google-adk 0.0.1 which is incompatible.
rag 0.1.0 requires google-auth<3.0.0,>=2.36.0, but you have google-auth 1.35.0 which is inc

In [2]:
from datasets import load_dataset

# Example: neural-bridge/rag-dataset-1200
dataset = load_dataset("neural-bridge/rag-dataset-1200")

# View train/test split
print(dataset.keys())     

# Inspect sample output
print(dataset["train"][0])

/Users/danukatheja/miniconda3/envs/adk_test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 240/240 [00:00<00:00, 25470.19 examples/s]

dict_keys(['train', 'test'])
{'context': 'Francisco Rogers found the answer to a search query collar george herbert essay\nLink ----> collar george herbert essay\nWrite my essay ESSAYERUDITE.COM\nconstitution research paper ideas\ndefinition essay humility\nbusiness strategy case study solution\ncorporals course essay\ndecisions in paradise essays\ncollege essay word count\ncredit cart terminal paper\nbyron don juan essay\ndemocratic party essays\ncoursework language learning material teaching\nchristmas commercialized essay\ndahrendorf essays theory society\nbuy apa format essay buy apa format essay\nconan doyle speckled band essay\ncollege essay application prompt\ncolumbia university mfa creative writing acceptance rate\ncrucible coursework questions\ncollege essay topics texas\ncover letter thesis proposal\nciting ma thesis\ncompare and contrast essays for elementary\ncoursework completed without degree\ncomparison islam christianity essay\ncheerleading stereotypes essay\ncultural 

In [3]:
dataset["train"].to_csv("rag_dataset_train.csv", index=False)
dataset["test"].to_csv("rag_dataset_test.csv", index=False)

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 63.46ba/s]


923350

In [4]:
dataset["train"]

Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 960
})

In [7]:
import pandas as pd
df_test =pd.read_csv("/Users/danukatheja/Downloads/rag-eval/rag-evals-starter/data/rag_dataset_test.csv")
df_test.head()

,context,question,answer
0,Trail Patrol Training\nWant to be a part of th...,What are some of the skills taught in the Trai...,The course teaches the essential skills necess...
1,"Lot Of Cbi Theater Ww2 Letters, 2 Newspapers, ...",Who was the original owner of the lot of items...,The original lot of items belonged to Lt. Neil...
2,Just.\nWe are a small all volunteer NGO Humani...,What is the main objective of Humanity Road as...,The main objective of Humanity Road is to 'clo...
3,One of two convicted killers who escaped from ...,Who were the two convicted killers that escape...,The two convicted killers that escaped from an...
4,(Continued from Part 1...)\n(Thirty years late...,Who was the person that came to help when Isaa...,Jesus was the person who came to help when Isa...


In [1]:
from datasets import load_dataset
import json
import pathlib

def hf_split_to_rag_jsonl(
    dataset_name: str,
    split: str,
    out_path: str,
    *,
    context_col: str = "context",
    question_col: str = "question",
    answer_col: str = "answer",
    include_q_and_a_in_content: bool = False,
    streaming: bool = False,
):
    """
    Streams the split (if streaming=True) and writes JSONL compatible with Vertex RAG Engine.
    """
    ds = load_dataset(dataset_name, split=split, streaming=streaming)

    out = pathlib.Path(out_path)
    out.parent.mkdir(parents=True, exist_ok=True)
    with out.open("w", encoding="utf-8") as f:
        for i, row in enumerate(ds):
            # robust getters
            ctx = str(row.get(context_col, "") or "")
            q = str(row.get(question_col, "") or "")
            a = str(row.get(answer_col, "") or "")
            rid = str(row.get("id") or f"{dataset_name.replace('/','_')}_{split}_{i:06d}")

            if include_q_and_a_in_content:
                content = f"Context:\n{ctx}\n\nQuestion:\n{q}\n\nAnswer:\n{a}"
                md = {"dataset": dataset_name, "split": split}
            else:
                content = ctx
                md = {
                    "question": q or None,
                    "answer": a or None,
                    "dataset": dataset_name,
                    "split": split,
                }
                md = {k: v for k, v in md.items() if v not in (None, "")}

            f.write(json.dumps({"id": rid, "content": content, "metadata": md}, ensure_ascii=False) + "\n")

# ---- Example usage (HF) ----
# hf_split_to_rag_jsonl("neural-bridge/rag-dataset-1200", "train", "out/hf_qa_train.jsonl",
#                       include_q_and_a_in_content=False, streaming=True)


/Users/danukatheja/miniconda3/envs/adk_test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# put this at the very top of your script
from datasets import config as ds_config
ds_config.TORCH_AVAILABLE = False  # force datasets to not import torch


In [4]:
# ---- Example usage (HF) ----
hf_split_to_rag_jsonl("neural-bridge/rag-dataset-1200", "train", "out/hf_qa_train.jsonl",
                      include_q_and_a_in_content=False, streaming=True)

In [ ]:
from google.cloud import storage

def upload_to_gcs(local_path: str, bucket: str, dest_path: str):
    client = storage.Client()
    blob = client.bucket(bucket).blob(dest_path)
    blob.upload_from_filename(local_path)
    return f"gs://{bucket}/{dest_path}"